In [3]:
import numpy as np
import astropy.units as u
import time
from datetime import datetime
today = int(datetime.today().strftime('%Y%m%d'))
import poppy
import skimage
from skimage.registration import phase_cross_correlation
import copy
from importlib import reload

import magpyx
from magpyx.utils import ImageStream
import purepyindi
from purepyindi import INDIClient
client0 = purepyindi.INDIClient('localhost', 7624)
client0.start()

import purepyindi2
from purepyindi2 import IndiClient
client = IndiClient()
client.connect()
client.get_properties()

from math_module import xp, xcipy, np_array
import utils
import fsm_utils
import cam_utils
from imshows import imshow1, imshow2, imshow3
from matplotlib.patches import Circle

v_bias = np.array([[50,50,50]]).T

wavelength = 633e-9
fl = 450e-3
fsm_pupil_diam = 7.1e-3
as_per_lamD = (wavelength/fsm_pupil_diam*u.radian).to(u.arcsec)
print(as_per_lamD)

pxscl_lamD = 3.45e-6 / (fl * wavelength/fsm_pupil_diam)
print(pxscl_lamD)


18.389524275269295 arcsec
0.08599262769878883


INFO:purepyindi2.transports:Connected to localhost:7624


In [ ]:
reload(cam_utils)
cam = cam_utils.CAM('campupil')


In [ ]:
npsf = 100
cam.set_roi(500, 875, npsf, client0)

In [ ]:
cam.set_exptime(0.0001, client0)

In [4]:
fsm = ImageStream('fsm')
fsm.write(v_bias)

In [13]:
amp = 0.5*as_per_lamD
v_offset = fsm_utils.get_fsm_volts(tip=amp, tilt=0*u.arcsec)
v_offset = fsm_utils.get_fsm_volts(tip=0*u.arcsec, tilt=amp)


Displacements: A = 5.31e-06 m, 4.85e-06 m, 4.85e-06 m. 
Delta Voltages: A = 53.09, B = 48.46, C = 48.46. 
Displacements: A = 5.00e-06 m, 5.27e-06 m, 4.73e-06 m. 
Delta Voltages: A = 50.00, B = 52.67, C = 47.33. 


In [14]:
volts = [v_bias, v_offset]
freq = 2
try:
    print('Modulating FSM ...')
    i = 0
    while True:
        fsm.write(volts[i%2])
        i += 1 
        time.sleep(1/(2*freq))
except KeyboardInterrupt:
    print('FSM modulation stopped!')
    fsm.write(0*v_bias)
    time.sleep(2)
    fsm.write(v_bias)
    time.sleep(0.25)

Modulating FSM ...
FSM modulation stopped!


In [43]:
fsm.write(0*v_bias)